<a href="https://colab.research.google.com/github/Manvi115/ML_Project1/blob/main/Fake_News_Prediction_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Roles of different libraries used by us:-

numpy is useful for creating numpy arrays

pandas is useful in creaing dataframs and storing the data in the dataframes

re is regular expression is used to search the words in a text

nltk is natural language toolkit .. corpus is text body...  stopwords means those words which doesn't add much value to text context like articles

stemming is a function which takes a word and removes suffix and prefix and returns the root word of it.

TfidVectorizer converts text into feature Vectors (numbers)

train_test_split splits data into training and testing data

LogisticRegression is our model


re is regular expression .. useful for searching text in a document
nltk is natural language tool kit 
corpus is the body of text
stemming is a function which removes a prefix and suffix from a word
TfidfVectorizer is used to convert text to features(numbers)
train_test_split seperates data into training and testing data
Logistic Regression is input


About the Dataset:


1.   id: unique id for a news article
2.   title: the title of a news article
3.   autor: author of the news article
4.   text: the text of the article; could be incomplete
5.   label: a label that marks whether the news article is real or fake:


> 1: Fake news
0: real news







In [4]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


downloading all the stopwords


In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
# printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Preprocessing

In [28]:
# loading the dataset to a pandas DataFrame becaues pandas dataframe is more strucutures table
news_dataset = pd.read_csv('/content/train.csv',error_bad_lines=False, engine="python")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 9885: unexpected end of data


In [29]:
news_dataset.shape


(9883, 5)

In [30]:
#print first five rows of dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [31]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

id          0
title     269
author    934
text       21
label       0
dtype: int64

In [32]:
#replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [33]:
#merging the author name and news title
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title']

In [34]:
print(news_dataset['content'])

0       Darrell Lucus House Dem Aide: We Didn’t Even S...
1       Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2       Consortiumnews.com Why the Truth Might Get You...
3       Jessica Purkiss 15 Civilians Killed In Single ...
4       Howard Portnoy Iranian woman jailed for fictio...
                              ...                        
9878    Warner Todd Huston ’White Devils’: Muhammad Al...
9879    ActivistPost We Now Have Proof Obamacare Was D...
9880    Meghan Moravcik Walbert After Losing a Foster ...
9881    Patrick Healy Hillary Clinton and Donald Trump...
9882    Alex Ansary Kremlin: NATO was Created for Agre...
Name: content, Length: 9883, dtype: object


In [35]:
#seperating the data & label
X = news_dataset.drop(columns='label', axis=1)  #for column 1 .. for row 0 
Y= news_dataset['label']

In [36]:
print(X)
print(Y)

        id                                              title  \
0        0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1        1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2        2                  Why the Truth Might Get You Fired   
3        3  15 Civilians Killed In Single US Airstrike Hav...   
4        4  Iranian woman jailed for fictional unpublished...   
...    ...                                                ...   
9878  9878  ’White Devils’: Muhammad Ali’s Racist Mosque S...   
9879  9879  We Now Have Proof Obamacare Was Designed to Fa...   
9880  9880  After Losing a Foster Child, Contemplating Ano...   
9881  9881  Hillary Clinton and Donald Trump, Ages 68 and ...   
9882  9882            Kremlin: NATO was Created for Agression   

                       author  \
0               Darrell Lucus   
1             Daniel J. Flynn   
2          Consortiumnews.com   
3             Jessica Purkiss   
4              Howard Portnoy   
...                  

STemming
It is the process of reducing a word to a root word .. we will only convert these words to vecors(numbers)
example: actor, actress, acting ---> act 

In [37]:
port_stem= PorterStemmer()

In [38]:
# # def to create  a function
def stemming(content): 
   stemmed_content = re.sub("[^a-zA-Z]"," ", content)  #(˄ means exclusion of everything which is not alphabets and replacing it with null space)
   stemmed_content = stemmed_content.lower() #converting everything to lower case to make every letter equal
   stemmed_content = stemmed_content.split() #to split data set in a list
   stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] #stemming
   stemmed_content = ' '.join(stemmed_content) #
   return stemmed_content 

In [39]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [40]:
print(news_dataset['content'])

0       darrel lucu hous dem aid even see comey letter...
1       daniel j flynn flynn hillari clinton big woman...
2                  consortiumnew com truth might get fire
3       jessica purkiss civilian kill singl us airstri...
4       howard portnoy iranian woman jail fiction unpu...
                              ...                        
9878    warner todd huston white devil muhammad ali ra...
9879              activistpost proof obamacar design fail
9880    meghan moravcik walbert lose foster child cont...
9881    patrick heali hillari clinton donald trump age...
9882                alex ansari kremlin nato creat agress
Name: content, Length: 9883, dtype: object


In [41]:
#separating the data and label
X= news_dataset['content'].values
Y= news_dataset['label'].values

In [42]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'meghan moravcik walbert lose foster child contempl anoth new york time'
 'patrick heali hillari clinton donald trump age share health detail new york time'
 'alex ansari kremlin nato creat agress']


In [43]:
print(Y)

[1 0 1 ... 0 0 1]


In [44]:
Y.shape 

(9883,)

In [45]:
#converting textual data to numerical data
vectorizer = TfidfVectorizer() #Tf stands for Term frequency .. idf stands for inverse document frequency ... It counts the number of times a word is repeated in a document
vectorizer.fit(X)

X= vectorizer.transform(X) #transform the values to numbers

In [ ]:
print(X)

  (0, 10391)	0.2857669667840401
  (0, 8918)	0.2565112117172494
  (0, 5910)	0.3487168483683741
  (0, 5727)	0.2903244395878411
  (0, 5119)	0.2516614676377858
  (0, 4671)	0.22403564420953384
  (0, 3322)	0.23636441963377403
  (0, 2553)	0.27121252902278775
  (0, 2422)	0.3487168483683741
  (0, 1986)	0.24613190933563625
  (0, 1672)	0.3853675512557288
  (0, 194)	0.2701946941978791
  (1, 11082)	0.29115814429141035
  (1, 4555)	0.18811594228359108
  (1, 3689)	0.7248137448454115
  (1, 2401)	0.26821911995681474
  (1, 1887)	0.18881311382112143
  (1, 1508)	0.37780536901010825
  (1, 1287)	0.1510080136288254
  (1, 1028)	0.28522299567265985
  (2, 10342)	0.42083360103298995
  (2, 6371)	0.49643466575246253
  (2, 3994)	0.3422741766307897
  (2, 3613)	0.38246297769831994
  (2, 2096)	0.4601213642359319
  :	:
  (8161, 10851)	0.38469983091692606
  (8161, 7247)	0.5214030038016889
  (8161, 4649)	0.6514605726286493
  (8162, 10607)	0.2545612841182937
  (8162, 8984)	0.36456157324510285
  (8162, 8863)	0.4158409531775

Splitting dataset into training and test data

In [46]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state =2) #label of X_train will be stored in Y_train .. stratify balances 0 and 1 ... random state is just to follow instructor.. it can be any integer value

Training the Model : LogisticRegression

In [47]:
model = LogisticRegression()

In [48]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation 

Accuracy Score

In [49]:
#accuracy score on the training data
X_train_prediction = model.predict(X_train) #X train prediction is predicted by our model and Y_train are original ... we will compare and get the accuracy of our model
training_data_accuracy = accuracy_score(X_train_prediction, Y_train) 

In [50]:
print("Accuracy score of the training data : ", training_data_accuracy)

Accuracy score of the training data :  0.9803946369845686


This model is preferred for binary classification

In [51]:
#accuracy score on testing data
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [52]:
print("Accuracy score of the testing data : ", testing_data_accuracy)

Accuracy score of the testing data :  0.9681335356600911


Making a Predictive System

In [53]:
X_new = X_test[10]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print("The news is Real")
else:
  print("The news is Fake")


[1]
The news is Fake


In [54]:
print(Y_test[10])

1
